加载环境变量

In [13]:
from dotenv import load_dotenv
import os
load_dotenv()
# 设置用户代理
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

1. 加载文档

In [14]:
from langchain_community.document_loaders import WebBaseLoader # pip install langchain-community

loader = WebBaseLoader(
    web_paths=("https://zh.wikipedia.org/wiki/黑神话：悟空",)
)
docs = loader.load()

2. 文档分块

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

3. 设置嵌入模型

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-zh-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

 4. 创建向量存储

In [17]:
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(all_splits)

['a2549b88-e928-4c93-92b9-5289514c1757',
 '5a9c30f6-4799-4f6e-9e34-90e93311e4fd',
 'ac7dbd8b-0e30-4346-9eb1-715d4c49050c',
 '1dc73e8d-f7ca-4722-aff2-03d282b413ca',
 '793a2319-169f-40ac-9ca7-b763a6d1bd6f',
 '3f01ed88-b791-4566-a1be-676c54890394',
 'fffc60e9-9dbf-4696-a041-da3a8033367e',
 '70b6eaa4-7bf8-4b55-a9e0-10a14d1011d6',
 '49bd7813-e5da-4562-a680-f4f94e2f51ca',
 'e49a292c-1df4-420c-acd0-bba3ef6e514f',
 '06497bfa-91cd-4d8b-91da-b3c958062006',
 'b780dcee-2341-4a7d-8078-808e94e7103e',
 '299a0e8c-f333-49c7-be4b-6c0116ccf835',
 '0ee97822-e906-45c3-90b7-54883f13b44c',
 '1fd2a951-18d6-425a-b6ed-99169ae50a69',
 '72e81f68-931e-47ea-862a-c10e815dfdb8',
 '506ab7ff-c218-4010-8e5c-084d4ac9f926',
 'f9805344-4dc7-4a76-9f56-7d5abd77c32d',
 '32e0a3ac-5c5a-4f65-a652-be361c9823de',
 'c201a295-7565-454f-a0d4-5414b553f082',
 '0e7f7d67-1c1e-463c-9db5-264cc704d0c3',
 'e2bda05c-04da-414e-8be7-705f2a7fa412',
 'dfdb9653-73be-49f8-9705-5a2191491d78',
 'd40ce28b-60b9-48ca-a2b6-2a542b1c49b2',
 'ceb2f3ad-2b31-

5. 构建用户查询

In [18]:
question = "黑悟空有哪些游戏场景？"

6. 在向量存储中搜索相关文档，并准备上下文内容

In [19]:
retrieved_docs = vector_store.similarity_search(question, k=3)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

7. 构建提示模板

In [20]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
                基于以下上下文，回答问题。如果上下文中没有相关信息，
                请说"我无法从提供的上下文中找到相关信息"。
                上下文: {context}
                问题: {question}
                回答:""")

8. 使用ollama本地大语言模型生成答案

In [21]:
from langchain_ollama import ChatOllama # pip install langchain-ollama
llm = ChatOllama(
    model="qwen2.5:3b",  # 可以根据需要更换其他模型，如 llama2, mistral 等
    request_timeout=300.0  # 增加超时时间
)

9. 生成答案

In [22]:
print(f"问题: {question}")
print("-"*50)
answer = llm.invoke(prompt.format(question=question, context=docs_content))
print(f"回答: {answer.content}")

问题: 黑悟空有哪些游戏场景？
--------------------------------------------------
回答: 《黑神话：悟空》中的游戏场景融合了多个中国著名的文化和自然地标，包括但不限于：

- 大足石刻（重庆）
- 小西天（山西）
- 南禅寺、铁佛寺、广胜寺等寺庙建筑
- 鹳雀楼

此外，游戏地图无法由玩家调整，但会随游戏进展而变化。游戏中的场景旨在提供一种沉浸式的中国古典文化体验，并通过这些地点传达游戏的故事背景和主题元素。
